In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

from flask import Flask, jsonify, request

In [2]:
tanah_dataset = pd.read_csv('dataset2.csv') 

In [3]:
# printing the first 5 rows of the dataset
tanah_dataset.head()

,Nitrogen,Fosfor,Potassium,Ph,SoilHumidity,SoilTemperature,SoilConductivity,Output
0,41,58,117,6.12,32.8,29.2,633,0
1,41,58,117,6.20,27.2,35.0,700,1
2,41,58,116,5.77,29.3,29.6,652,0
3,29,40,81,6.50,29.5,29.2,600,3
4,41,58,116,35.10,23.4,24.0,649,2


In [4]:
tanah_dataset.shape

(51, 8)

In [5]:
# getting the statistical measures of the data
tanah_dataset.describe()

,Nitrogen,Fosfor,Potassium,Ph,SoilHumidity,SoilTemperature,SoilConductivity,Output
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,49.921569,44.117647,106.019608,6.960980,29.056863,26.480392,548.333333,1.294118
std,39.237147,13.661108,33.821585,4.047515,3.029076,6.538961,136.045532,1.063844
min,16.000000,20.000000,49.000000,5.500000,23.000000,6.300000,131.000000,0.000000
25%,28.500000,36.000000,77.000000,6.205000,28.350000,22.600000,488.000000,0.000000
50%,39.000000,40.000000,111.000000,6.340000,29.100000,27.800000,588.000000,1.000000
75%,41.000000,56.000000,116.500000,6.500000,29.700000,29.400000,649.000000,2.000000
max,170.000000,79.000000,175.000000,35.100000,46.000000,49.000000,719.000000,3.000000


In [6]:
tanah_dataset['Output'].value_counts()
# 0 -> Non-Fiertic
# 1 -> Fiertic

1    17
0    14
2    11
3     9
Name: Output, dtype: int64

In [7]:
tanah_dataset.groupby('Output').mean()

,Nitrogen,Fosfor,Potassium,Ph,SoilHumidity,SoilTemperature,SoilConductivity
Output,,,,,,,
0,38.142857,54.214286,108.285714,6.515714,29.228571,24.064286,537.142857
1,42.647059,47.823529,100.647059,6.507647,29.029412,27.023529,583.823529
2,53.454545,34.909091,104.545455,8.832727,29.009091,28.290909,551.090909
3,77.666667,32.666667,114.444444,6.222222,28.900000,27.000000,495.333333


In [8]:
# separating the data and labels
X = tanah_dataset.drop(columns = 'Output', axis=1)
Y = tanah_dataset['Output']

In [9]:
print(X)

    Nitrogen  Fosfor  Potassium     Ph  SoilHumidity  SoilTemperature  \
0         41      58        117   6.12          32.8             29.2   
1         41      58        117   6.20          27.2             35.0   
2         41      58        116   5.77          29.3             29.6   
3         29      40         81   6.50          29.5             29.2   
4         41      58        116  35.10          23.4             24.0   
5         40      56        114   6.41          30.5             30.2   
6         29      41         81   6.14          28.5             35.4   
7         26      37         74   6.50          31.2             28.7   
8         40      56        114   7.00          29.1             24.9   
9         42      59        118   6.26          29.0              6.3   
10        25      37         73   5.89          29.9             31.0   
11        17      24         49   7.00          29.9             20.0   
12        27      37         75   6.44          29.

In [10]:
print(Y)

0     0
1     1
2     0
3     3
4     2
5     0
6     1
7     2
8     1
9     1
10    2
11    0
12    2
13    3
14    0
15    0
16    1
17    0
18    1
19    1
20    0
21    0
22    1
23    2
24    1
25    1
26    3
27    1
28    1
29    2
30    3
31    0
32    0
33    1
34    2
35    3
36    3
37    1
38    0
39    2
40    1
41    0
42    0
43    1
44    1
45    3
46    2
47    2
48    2
49    3
50    3
Name: Output, dtype: int64


In [11]:
scaler = StandardScaler()

In [12]:
scaler.fit(X)

StandardScaler()

In [13]:
standardized_data = scaler.transform(X)

In [14]:
print(standardized_data)

[[-2.29638067e-01  1.02630686e+00  3.27886836e-01 -2.09844429e-01
   1.24803181e+00  4.20046760e-01  6.28533262e-01]
 [-2.29638067e-01  1.02630686e+00  3.27886836e-01 -1.89882544e-01
  -6.19112690e-01  1.31586386e+00  1.12591588e+00]
 [-2.29638067e-01  1.02630686e+00  2.98025714e-01 -2.97177679e-01
   8.10664979e-02  4.81827250e-01  7.69582065e-01]
 [-5.38513884e-01 -3.04413051e-01 -7.47113577e-01 -1.15025473e-01
   1.47750230e-01  4.20046760e-01  3.83553763e-01]
 [-2.29638067e-01  1.02630686e+00  2.98025714e-01  7.02134862e+00
  -1.88610360e+00 -3.83099605e-01  7.47311202e-01]
 [-2.55377718e-01  8.78449089e-01  2.38303468e-01 -1.37482594e-01
   4.81168891e-01  5.74497984e-01  4.72637217e-01]
 [-5.38513884e-01 -2.30484167e-01 -7.47113577e-01 -2.04853958e-01
  -1.85668431e-01  1.37764435e+00  6.87922232e-01]
 [-6.15732838e-01 -5.26199702e-01 -9.56141435e-01 -1.15025473e-01
   7.14561954e-01  3.42821148e-01  1.20015210e+00]
 [-2.55377718e-01  8.78449089e-01  2.38303468e-01  9.73631183e-0

In [15]:
X = standardized_data
Y = tanah_dataset['Output']

In [16]:
print(X)
print(Y)

[[-2.29638067e-01  1.02630686e+00  3.27886836e-01 -2.09844429e-01
   1.24803181e+00  4.20046760e-01  6.28533262e-01]
 [-2.29638067e-01  1.02630686e+00  3.27886836e-01 -1.89882544e-01
  -6.19112690e-01  1.31586386e+00  1.12591588e+00]
 [-2.29638067e-01  1.02630686e+00  2.98025714e-01 -2.97177679e-01
   8.10664979e-02  4.81827250e-01  7.69582065e-01]
 [-5.38513884e-01 -3.04413051e-01 -7.47113577e-01 -1.15025473e-01
   1.47750230e-01  4.20046760e-01  3.83553763e-01]
 [-2.29638067e-01  1.02630686e+00  2.98025714e-01  7.02134862e+00
  -1.88610360e+00 -3.83099605e-01  7.47311202e-01]
 [-2.55377718e-01  8.78449089e-01  2.38303468e-01 -1.37482594e-01
   4.81168891e-01  5.74497984e-01  4.72637217e-01]
 [-5.38513884e-01 -2.30484167e-01 -7.47113577e-01 -2.04853958e-01
  -1.85668431e-01  1.37764435e+00  6.87922232e-01]
 [-6.15732838e-01 -5.26199702e-01 -9.56141435e-01 -1.15025473e-01
   7.14561954e-01  3.42821148e-01  1.20015210e+00]
 [-2.55377718e-01  8.78449089e-01  2.38303468e-01  9.73631183e-0

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=2)

In [18]:
print(X.shape, X_train.shape, X_test.shape)

(51, 7) (40, 7) (11, 7)


In [19]:
classifier = svm.SVC(kernel='linear')

In [20]:
#training the support vector Machine Classifier
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

In [21]:
# accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [22]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.575


In [23]:
# accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [24]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.5454545454545454


In [22]:
input_data = (
    0,
        700,
        0,
        0,
        1,
        274,
        0
)

# changing the input_data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

# reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

# standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0] == 0):
  print('Buruk')
if (prediction[0] == 1):
  print('Kurang Subur')
if (prediction[0] == 2):
  print('Subur')
if (prediction[0] == 3):
  print('Sangat Subur')

[[-1.28496378 48.48865023 -3.16586451 -1.73692868 -9.35468161 38.22970641
  -4.07061896]]
[0]
Buruk


C:\Users\rizqi\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\rizqi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
